In [48]:
import tensorflow as tf
import os
from os.path import isfile, join
import numpy as np
import shutil
from pathlib import Path
from IPython.display import display, Audio
import subprocess
import librosa
import soundfile as sf
from keras.layers import Conv1D
import keras
import joblib
import librosa

data_directory = "Speech-processing\\Keras-speaker-recognisition"
audio_folder = "audio"
noise_folder = "noise"

audio_path = os.path.join(data_directory, audio_folder)
noise_path = os.path.join(data_directory, noise_folder)

valid_split = 0.1

shuffle_seed = 43

# Sample rate, also known as sampling rate, refers to the number of samples of audio that are taken per second 
# when digitizing or recording an analog audio signal. It is measured in Hertz (Hz) and 
# typically expressed in kilohertz (kHz) or megahertz (MHz).

sample_rate = 16000

scale = 0.5

batch_size = 128

epochs = 15


In [2]:
for folder in os.listdir(data_directory):
    if os.path.isdir(os.path.join(data_directory, folder)):
        if folder in [audio_folder, noise_folder]:
            
            continue
        elif folder in ["other", "_background_noise_"]:
            
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(noise_path, folder),
            )
        else:
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(audio_path, folder),
            )

noise_paths = []
for subdir in os.listdir(noise_path):
    subdir_path = Path(noise_path) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

In [26]:
command = (
    "for dir in `ls -1 " + noise_path + "`; do "
    "for file in `ls -1 " + noise_path + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)

for path in noise_paths:
    audio_file, sr = librosa.load(path, sr=None)
    audio_resampled = librosa.resample(audio_file, sr, 16000)
    sf.write(path, audio_resampled, 16000)

C:\Users\Atharva\AppData\Local\Temp\ipykernel_35168\3626655206.py:15: FutureWarning: Pass orig_sr=16000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  audio_resampled = librosa.resample(audio_file, sr, 16000)
C:\Users\Atharva\AppData\Local\Temp\ipykernel_35168\3626655206.py:15: FutureWarning: Pass orig_sr=22050, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  audio_resampled = librosa.resample(audio_file, sr, 16000)
C:\Users\Atharva\AppData\Local\Temp\ipykernel_35168\3626655206.py:15: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  audio_resampled = librosa.resample(audio_file, sr, 16000)


In [27]:
os.system(command)

def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == sample_rate:
        slices = int(sample.shape[0] / sample_rate)
        sample = tf.split(sample[: slices * sample_rate], slices)
        return sample
    else:
        print("Sampling rate for",path, "is incorrect")
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

In [28]:
noises

<tf.Tensor: shape=(354, 16000, 1), dtype=float32, numpy=
array([[[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        ...,
        [-0.00747681],
        [ 0.00854492],
        [ 0.03442383]],

       [[ 0.02334595],
        [ 0.02230835],
        [ 0.03738403],
        ...,
        [ 0.04589844],
        [ 0.01644897],
        [-0.0062561 ]],

       [[ 0.02755737],
        [ 0.00061035],
        [-0.02462769],
        ...,
        [ 0.03149414],
        [ 0.00411987],
        [ 0.02502441]],

       ...,

       [[-0.17086792],
        [-0.01544189],
        [ 0.01107788],
        ...,
        [-0.01834106],
        [-0.0920105 ],
        [-0.04959106]],

       [[ 0.02841187],
        [ 0.0244751 ],
        [-0.05978394],
        ...,
        [-0.11184692],
        [-0.06658936],
        [ 0.09091187]],

       [[-0.06607056],
        [ 0.03790283],
        [ 0.00744629],
        ...,
        [ 0.00509644],
        [-0.07342529],
        [ 0.10510254]]], dtype=float3

In [29]:
def path_to_audio(path):
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, sample_rate)
    return audio

def paths_and_labels_to_dataset(audio_paths, labels):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))

def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        audio = audio + noise * prop * scale

    return audio

def audio_to_fft(audio):
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])

In [30]:
class_names = os.listdir(audio_path)
print(class_names,)

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Speaker:",(name))
    dir_path = Path(audio_path) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

['Benjamin_Netanyau', 'Jens_Stoltenberg', 'Julia_Gillard', 'Magaret_Tarcher', 'Nelson_Mandela']
Speaker: Benjamin_Netanyau
Speaker: Jens_Stoltenberg
Speaker: Julia_Gillard
Speaker: Magaret_Tarcher
Speaker: Nelson_Mandela


In [31]:
# Shuffle to generate random data
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(audio_paths)
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(labels)

# Split into training and validation
num_val_samples = int(valid_split * len(audio_paths))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]


valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

# Create datasets, one for training and the other for validation
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=shuffle_seed).batch(32)

In [32]:
valid_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 16000, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [33]:
# Add noise to the training set
train_ds = train_ds.map(
    lambda x, y: (add_noise(x, noises, scale=scale), y),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
)

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)

train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [38]:
from keras import layers
from keras import callbacks


def residual_block(x, filters, conv_num = 3, activation = "relu"):
    s = layers.Conv1D(filters, 1, padding = "same")(x)
    
    for i in range(conv_num - 1):
        x = layers.Conv1D(filters, 3, padding = "same")(x)
        x = layers.Activation(activation)(x)
    
    x = layers.Conv1D(filters, 3, padding = "same")(x)
    x = layers.Add()([x, s])
    x = layers.Activation(activation)(x)
    
    return layers.MaxPool1D(pool_size = 2, strides = 2)(x)

def build_model(input_shape, num_classes):
    inputs = layers.Input(shape = input_shape, name = "input")
    
    x = residual_block(inputs, 16, 2)
    x = residual_block(inputs, 32, 2)
    x = residual_block(inputs, 64, 3)
    x = residual_block(inputs, 128, 3)
    x = residual_block(inputs, 128, 3)
    x = layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(x)
    
    outputs = layers.Dense(num_classes, activation = "softmax", name = "output")(x)
    
    return keras.models.Model(inputs = inputs, outputs = outputs)

model = build_model((sample_rate // 2, 1), len(class_names))

model.summary()

model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) 

model_save_filename = "model.h5"

earlystopping_cb = callbacks.EarlyStopping(patience=10, restore_best_weights=True)

mdlcheckpoint_cb = callbacks.ModelCheckpoint(model_save_filename, monitor="val_accuracy", save_best_only=True)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 8000, 1)]    0           []                               
                                                                                                  
 conv1d_33 (Conv1D)             (None, 8000, 128)    512         ['input[0][0]']                  
                                                                                                  
 activation_23 (Activation)     (None, 8000, 128)    0           ['conv1d_33[0][0]']              
                                                                                                  
 conv1d_34 (Conv1D)             (None, 8000, 128)    49280       ['activation_23[0][0]']          
                                                                                            

In [41]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/15
53/53 [==============================] - 197s 4s/step - loss: 2.4096 - accuracy: 0.7000 - val_loss: 0.2334 - val_accuracy: 0.9267
Epoch 2/15
53/53 [==============================] - 195s 4s/step - loss: 0.1703 - accuracy: 0.9363 - val_loss: 0.1395 - val_accuracy: 0.9453
Epoch 3/15
53/53 [==============================] - 197s 4s/step - loss: 0.1484 - accuracy: 0.9412 - val_loss: 0.1194 - val_accuracy: 0.9587
Epoch 4/15
53/53 [==============================] - 194s 4s/step - loss: 0.0876 - accuracy: 0.9693 - val_loss: 0.0881 - val_accuracy: 0.9680
Epoch 5/15
53/53 [==============================] - 191s 4s/step - loss: 0.1203 - accuracy: 0.9570 - val_loss: 0.1179 - val_accuracy: 0.9627
Epoch 6/15
53/53 [==============================] - 189s 4s/step - loss: 0.0746 - accuracy: 0.9747 - val_loss: 0.0862 - val_accuracy: 0.9680
Epoch 7/15
53/53 [==============================] - 195s 4s/step - loss: 0.0518 - accuracy: 0.9815 - val_loss: 0.0784 - val_accuracy: 0.9747
Epoch 8/15
53

In [46]:
print("Accuracy of model:", model.evaluate(valid_ds))

24/24 [==============================] - 7s 271ms/step - loss: 0.0527 - accuracy: 0.9787
Accuracy of model: [0.05274485424160957, 0.9786666631698608]


In [49]:
joblib.dump(model, 'garima-ahmed-model.joblib')

INFO:tensorflow:Assets written to: ram://3c6e312d-0e46-41e9-a5fe-a9c5d357c36d/assets


INFO:tensorflow:Assets written to: ram://3c6e312d-0e46-41e9-a5fe-a9c5d357c36d/assets


['garima-ahmed-model.joblib']

In [43]:
SAMPLES_TO_DISPLAY = 10

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=scale), y))

for audios, labels in test_ds.take(1):
    ffts = audio_to_fft(audios)
    y_pred = model.predict(ffts)
    rnd = np.random.randint(0, batch_size, SAMPLES_TO_DISPLAY)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(SAMPLES_TO_DISPLAY):
        print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[labels[index]],
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[y_pred[index]],
            )
        )
        if labels[index] ==y_pred[index]:
            print("Welcome")
        else:
            print("Sorry")
        print("The speaker is" if labels[index] == y_pred[index] else "", class_names[y_pred[index]])

4/4 [==============================] - 1s 290ms/step
Speaker: Julia_Gillard	Predicted: Julia_Gillard
Welcome
The speaker is Julia_Gillard
Speaker: Julia_Gillard	Predicted: Julia_Gillard
Welcome
The speaker is Julia_Gillard
Speaker: Jens_Stoltenberg	Predicted: Jens_Stoltenberg
Welcome
The speaker is Jens_Stoltenberg
Speaker: Nelson_Mandela	Predicted: Nelson_Mandela
Welcome
The speaker is Nelson_Mandela
Speaker: Benjamin_Netanyau	Predicted: Benjamin_Netanyau
Welcome
The speaker is Benjamin_Netanyau
Speaker: Benjamin_Netanyau	Predicted: Benjamin_Netanyau
Welcome
The speaker is Benjamin_Netanyau
Speaker: Jens_Stoltenberg	Predicted: Jens_Stoltenberg
Welcome
The speaker is Jens_Stoltenberg
Speaker: Nelson_Mandela	Predicted: Nelson_Mandela
Welcome
The speaker is Nelson_Mandela
Speaker: Benjamin_Netanyau	Predicted: Benjamin_Netanyau
Welcome
The speaker is Benjamin_Netanyau
Speaker: Julia_Gillard	Predicted: Julia_Gillard
Welcome
The speaker is Julia_Gillard


In [53]:
def paths_to_dataset(audio_paths):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    return tf.data.Dataset.zip((path_ds))

def predict(path, labels):
    test = paths_and_labels_to_dataset(path, labels)


    test = test.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
    )
    test = test.prefetch(tf.data.experimental.AUTOTUNE)


    test = test.map(lambda x, y: (add_noise(x, noises, scale=scale), y))

    for audios, labels in test.take(1):
        ffts = audio_to_fft(audios)
        y_pred = model.predict(ffts)
        rnd = np.random.randint(0, 1, 1)
        audios = audios.numpy()[rnd, :]
        labels = labels.numpy()[rnd]
        y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(1):
            # print(
            # "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
            # "[92m",y_pred[index],
            #     "[92m", y_pred[index]
            #     )
            # )
            
            print("Speaker Predicted:",class_names[y_pred[index]])

In [55]:
path = ["audio\\Benjamin_Netanyau\\25.wav"]
labels = ["unknown"]
try:
    predict(path, labels)
except:
    print("Error! Check if the file correctly passed or not!")

1/1 [==============================] - 0s 40ms/step
Speaker Predicted: Benjamin_Netanyau
